# Composición de Prolog, o ¿cómo monto yo esta cosa?
Vamos a recapitular un momento.<br>
Hemos visto qué componentes tiene de fondo Prolog.<br>
Hemos visto los elementos básicos de Prolog, y como escribir predicados básicos.<br>
Acabamos de aprender como utilizar consultas en Prolog.<br>
¿Qué nos falta?<br>
~~Llorar~~ **Practicar**.<br>

Vamos a escribir tu primer programa en Prolog. No te preocupes, ¡estaré a tu lado y el búho promete no reírse!<br>
Imagina que una empresa de márketing nos pide ayuda para organizar su agenda de clientes y determinar potenciales compradores de sus nuevos y demoledores productos, el **PeinaPelos 3000X** y el **AbrillantaBombillas Turbo 5000**. En esta empresa, por desgracia, los vendedores no tienen el suficiente carisma como para venderle un peine a un calvo y agradecerían tener un sistema que automatice el proceso de selección de clientes para las dos secciones de ventas, una especializada en productos para pelo y la otra en productos para pelones en función del departamento del vendedor y el tipo de cliente.<br>
Los datos que disponemos son los siguientes:<br>
Ramon y Javier trabajan en el departamento de melenas.<br>
Alberto y Sara trabajan en el departamento de bombillas.<br>
Actualmente sabemos que Melinda, Ana y Bruno tienen pelo Pantene, mientras que Guillermo, Álvaro y Antonio cuentan con un tremendo tobogán de piojos.<br>
Te he adelantado la tarea un poco, tengo hecho esto:<br>
% Departamentos y empleados
trabaja_en(javier, melenas).
trabaja_en(alberto, bombillas).

% Clientes
tiene_pelo(ana).
tiene_pelo(bruno).

no_tiene_pelo(alvaro).
no_tiene_pelo(antonio).

% Regla para determinar si un vendedor puede vender productos a un cliente con pelo
puede_vender(Vendedor, Cliente) :-
    trabaja_en(Vendedor, melenas),
    tiene_pelo(Cliente).




Solo faltaría añadir a nuestra base de conocimiento a Ramón, Sara, Melinda y Guillermo. ¿Podrías crear los hechos que faltan? Que me cierra el Mercadona y no me queda hummus.<br>

#### Ponerlo como test







¡Perfecto, bien hecho! Yo tengo hummus y tú tienes la base de hechos terminada. Me acabo de dar cuenta de que falta una regla para verificar que los vendedores del AbrillantaBombillas están tratando con el cliente correcto. ¿Podrías rellenar lo que falta cogiendo la otra regla puede_vender como ejemplo? 

        puede_vender(Vendedor, Cliente) :-
            trabaja_en(Vendedor, _______),
            no_tiene_pelo(______).

In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output

quizzes = [
    {
        'question': '¿Qué falta en la segunda línea?',
        'options': ['bombilla', 'melenas', 'tiene_pelo', 'Cliente'],
        'correct_option': 'bombilla'
    },
        {
        'question': '¿Qué falta en la tercera línea?',
        'options': ['melenas', 'Vendedor', 'Cliente', 'puede_vender'],
        'correct_option': 'Cliente'
    }
]

current_question_index = 0

out = widgets.Output()

def create_quiz():
    global current_question_index
    
    with out:
        clear_output(wait=True)
        
        if current_question_index < len(quizzes):
            quiz = quizzes[current_question_index]
            question_label = widgets.Label(value=quiz['question'])
            buttons = [widgets.Button(description=option) for option in quiz['options']]
            feedback_label = widgets.Label(value='')

            def on_button_clicked(b):
                if b.description == quiz['correct_option']:
                    feedback_label.value = '¡Correcto!'
                    next_question()
                else:
                    feedback_label.value = 'Incorrecto, intenta de nuevo.'
            
            for button in buttons:
                button.on_click(on_button_clicked)
            
            display(widgets.VBox([question_label] + buttons + [feedback_label]))
        else:
            display(widgets.Label(value='¡Enhorabuena, máquina!'))

def next_question():
    global current_question_index
    current_question_index += 1
    create_quiz()

create_quiz()

display(out)



¡Genial! Ahora podemos hacer las pruebas que queramos y verificar que está todo correcto:<br>

In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
from swiplserver import PrologMQI

output_box_prolog = widgets.Output()
input_query_prolog = widgets.Text(
    placeholder='Escribe aquí...',
    description='Consulta:',
    disabled=False,
    continuous_update=False
)

error_count_prolog = 0

def show_error_message(user_input_prolog):
    global error_count_prolog
    with output_box_prolog:
        output_box_prolog.clear_output()
        if error_count_prolog < 2:
            message = "Revisa la sintaxis, por favor."
        elif error_count_prolog < 4:
            message = "Mira bien, ¿se te ha olvidado el punto al final? ¿El nombre del predicado es correcto?"
        elif error_count_prolog < 6:
            message = "Estás haciendo al búho llorar."
        elif error_count_prolog < 7:
            message = "¿Te diviertes?"
        else:
            message = "..."
        display(HTML(f'<div style="text-align: center;">{message}</div>'))
        error_count_prolog += 1

def on_key_press_prolog(event):
    global error_count_prolog
    user_input_prolog = input_query_prolog.value.strip()
    
    with PrologMQI() as mqi:
        with mqi.create_thread() as prolog_thread:
            prolog_thread.query("set_prolog_flag(encoding,utf8).")
            try:
                prolog_thread.query("consult(\"programas/ventapelosnopelos.pl\")")
                query_prolog = user_input_prolog
                prolog_thread.query_async(query_prolog, find_all=True)
                result_prolog = prolog_thread.query_async_result()
                
                with output_box_prolog:
                    output_box_prolog.clear_output()
                    if not result_prolog:
                        show_error_message(user_input_prolog)
                    else:
                        error_count_prolog = 0  
                        print(f"¡Ole, bien hecho!")
            
            except Exception as e:
                with output_box_prolog:
                    output_box_prolog.clear_output()
                    show_error_message(user_input_prolog)

def on_button_clicked_prolog(_):
    input_container_prolog.children = [input_query_prolog]

input_query_prolog.observe(on_key_press_prolog, names='value')

input_container_prolog = widgets.VBox(
    [input_query_prolog, output_box_prolog],
    layout=widgets.Layout(align_items='center')  
)

display(input_container_prolog)


Me acaban de llamar de la empresa, su productividad ha subido un 35% y la horda de hippies peludos que les monta una manifestación cada vez que le ofrecían crema de calva a un melenas ha levantado el piquete semipermanente que tenían a la entrada de sus oficinas. ¡¡Buen trabajo!!<br>
Han quedado muy contentos.<br>
Tremendamente contentos.<br>
Tan contentos que se ha corrido la voz sobre las maravillas de Prolog y ahora mismo tengo otros 15 proyectos encima de la mesa. Tienes dos minutos para coger un café, **hay que trabajar**.

Vamos con nuestro siguiente proyecto. Un ayuntamiento del estado de Alabama, cansado de líos entre árboles genealógicos, desea automatizar el sistema y detectar casos **peculiares** entre cónyuges para aliviar la carga de trabajo del personal laboral. Por suerte, ya nos han adelantado la base de conocimiento con todos los miembros del condado, así que solo nos falta diseñar las reglas para determinar si dos individuos pueden casarse sin consecuencias **catastróficas**. Aquí tenemos unos pocos casos de muestra:<br>

    % Primera generación
    padre(ferdinand,john).
    padre(ferdinand,james).
    
    % Segunda generación
    
    padre(john, harry).       
    madre(mary, harry).
    padre(john, william).     
    madre(mary, william).
    
    padre(james, emily).      
    madre(susan, emily).
    padre(james, margaret).
    madre(susan, margaret).
    padre(james, oliver).
    madre(susan, oliver).
    
    % Tercera generación
    padre(harry, jim).       
    madre(emily, jim).
    padre(harry, sophie).
    madre(emily, sophie).
    
    padre(william, sebastian).
    madre(margaret, sebastian).
    padre(william, charlotte).
    madre(margaret, charlotte).
    
    padre(oliver, george).    
    madre(emma, george).
    padre(oliver, lily).
    madre(emma, lily).
    
    padre(sebastian, ella).       
    madre(claire, ella).
    padre(sebastian, jack).
    madre(claire, jack).

Te he quitado parte del curro y he definido todas las relaciones familiares, pero falta establecer las reglas. Desde el ayuntamiento nos dicen que no pueden casarse dos personas con tercer grado de consanguinidad o menos(Padres e hijos, hermanos, abuelos y nietos, tíos y sobrinos) pero además, **por si acaso**, quieren prohibir el matrimonio entre primos cuyos padres también sean primos. Acaba lo que me queda que yo tengo que salir un segundo a por tabaco. <br>grado3(P1, P2) :- padre(P1, P2).

    grado3(P1, P2) :- padre(P1, P2).
    grado3(P1, P2) :- madre(P1, P2).
    grado3(P1, P2) :- hermano(P1, P2).
    grado3(P1, P2) :- ¿?(P1, P2).
    grado3(P1, P2) :- abuela(P1, P2).
    grado3(P1, P2) :- ¿?(P1, P2).
    grado3(P1, P2) :- tia(P1, P2).

    no_matrimonio_primos(P1, P2) :-
    ¿?(P1, P2),
    padre(¿?, P1),
    padre(¿?, P2),
    primo(Padre1, Padre2).

    no_matrimonio_primos(P1, P2) :-
    primo(P1, P2),
    ¿?(Madre1, P1),
    ¿?(Madre2, P2),
    primo(¿?, ¿?).

Test final

### - [Siguiente apartado - Operadores de Prolog](Operadores.ipynb)
### - [Apartado anterior - Posología de Prolog](Predicados.ipynb)
### - [Volver al índice](../Indice.ipynb)